In [1]:
import pandas as pd
pd.get_option("display.max_columns")

20

Loading the dataset

In [2]:
data=pd.read_csv("https://www.treasury.gov/ofac/downloads/alt.csv",header=None)

In [3]:
data.columns=['Group','id','extra','Input','extra2']
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13598 entries, 0 to 13597
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Group   13598 non-null  object 
 1   id      13597 non-null  float64
 2   extra   13597 non-null  object 
 3   Input   13597 non-null  object 
 4   extra2  13597 non-null  object 
dtypes: float64(1), object(4)
memory usage: 531.3+ KB


In [4]:
data.head(100)

,Group,id,extra,Input,extra2
0,36,12.0,aka,AERO-CARIBBEAN,-0-
1,173,57.0,aka,AVIA IMPORT,-0-
2,306,220.0,aka,NATIONAL BANK OF CUBA,-0-
3,540,471.0,aka,COIBA,-0-
4,552,475.0,aka,CRYMSA,-0-
...,...,...,...,...,...
95,4697,3917.0,aka,ISLAMIC JIHAD ORGANIZATION,-0-
96,4697,3918.0,aka,REVOLUTIONARY JUSTICE ORGANIZATION,-0-
97,4697,3919.0,aka,ORGANIZATION OF THE OPPRESSED ON EARTH,-0-
98,4697,3920.0,aka,ISLAMIC JIHAD FOR THE LIBERATION OF PALESTINE,-0-


In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

Checking for null values and dropping the rows which contain nulls

In [8]:
data.isna().sum()

Group     0
id        1
extra     1
Input     1
extra2    1
dtype: int64

In [9]:
data.dropna(inplace=True)

In [10]:
import time

Creating a dictionary below which takes the Group as the dictionary key and assigns the input values to to the keys

In [13]:
import collections
key1=collections.defaultdict(int)
for i in range(0,len(data)):
  key1[data.iloc[i,3]]=data.iloc[i,0]

In [14]:
group=collections.defaultdict(list)
for i in range(0,len(data)):
  group[data.iloc[i,0]].append(data.iloc[i,3])
print(group)  

defaultdict(<class 'list'>, {'36': ['AERO-CARIBBEAN'], '173': ['AVIA IMPORT'], '306': ['NATIONAL BANK OF CUBA'], '540': ['COIBA'], '552': ['CRYMSA'], '555': ['COPROVA', 'COPROVA SARL'], '559': ['CIMEX'], '651': ['DELVEST HOLDING COMPANY'], '735': ['CUBANA AIRLINES'], '751': ['CARIBBEAN EXPORT ENTERPRISE'], '758': ['CUBATUR'], '815': ['GALAX INC.'], '906': ['HAVANA INTERNATIONAL BANK, LTD'], '1287': ['CUFLET', 'CUBAN FREIGHT ENTERPRISE'], '2109': ['TROVER, S.A.'], '2110': ['TROPICANA TOURS GMBH'], '2674': ['ZAYDAN, Muhammad'], '2676': ['AL-ZAWAHIRI, Ayman', 'SALIM, Ahmad Fuad', 'AL-ZAWAHIRI, Aiman Muhammad Rabi'], '2677': ['ZUMAR, Abbud', 'AL-ZUMAR, Abbud', 'EL-ZOMOR, Aboud Abdul Latif Hasan', 'AL-ZUMUR, Abood Abdul Latif Hassan', 'AL-ZAMUR, Abboud Abdul Latif Hassan'], '2681': ['HAWATMEH, Nayif', 'HAWATMAH, Nayif', 'KHALID, Abu'], '2683': ['JIBRIL, Ahmad'], '2686': ['NASRALLAH, Hasan Abd-al-Karim'], '2831': ['MIDCO FINANCIAL S.A.', 'MONTANA MANAGEMENT INC.'], '3749': ['GENERAL NICKEL S

### FUZZ RATIO

We will deploy fuzz match for the first 20 rows below. We will check each of these 20 rows against all of the other rows in the dataset. The output will have 2 columns- the first containing the input we test. The second will contain a list of the other entries which have have a fuzzy match ratio above 80 with the entry in the first column. Each element in the second column is in the format- ratio, the entry and the group the entry belongs to

In [17]:
#Fuzz Ratio
import heapq
threshold=80
ans=[]
vis=set()
start = time.time()
for i in range (0,20):
  h=[]
  if data.iloc[i,3] in vis:
    continue
  vis.add(data.iloc[i,3])  
  for j in range(0,len(data)):
    if data.iloc[j,3] in vis:
      continue
    val=fuzz.ratio(data.iloc[i,3].lower(),data.iloc[j,3].lower())
    heapq.heappush(h,(-val,data.iloc[j,3],data.iloc[j,0]))
  c=0
  p=[]
  p.append(data.iloc[i,3]+'; '+data.iloc[i,0])
  c1=[]
  while(h):
    c+=1
    val1,val2,val3=heapq.heappop(h)
    val1=-val1
    if val1<threshold:
      break
    vis.add(val2)
    c1.append(str(val1)+"; "+ val2+ "; " +val3)
  p.append(c1)
  ans.append(p)
end = time.time()
print("The time of execution of above program is :", end-start)



The time of execution of above program is : 55.21203088760376


In [18]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.set_option('display.max_colwidth', None)
fuzz_ratio=pd.DataFrame(ans)
fuzz_ratio.head(100)

,0,1
0,AERO-CARIBBEAN; 36,[]
1,AVIA IMPORT; 173,[]
2,NATIONAL BANK OF CUBA; 306,[86; NATIONAL BANK OF IRAN; 25578]
3,COIBA; 540,[]
4,CRYMSA; 552,[]
5,COPROVA; 555,[]
6,COPROVA SARL; 555,[]
7,CIMEX; 559,[]
8,DELVEST HOLDING COMPANY; 651,[]
9,CUBANA AIRLINES; 735,[]


### FUZZ PARTIAL RATIO

The partial ratio() function allows us to perform substring matching. This works by taking the shortest string and matching it with all the substrings that are of the equal length.

FuzzyWuzzy also has more powerful functions to help with matching strings in more complex situations. 

We apply the same procedure that we applied for Fuzz Ratio. 

In [19]:
#Fuzz Partial Ratio
import heapq
threshold=80
ans=[]
vis=set()
start = time.time()
for i in range (0,20):
  h=[]
  if data.iloc[i,3] in vis:
    continue
  vis.add(data.iloc[i,3])  
  for j in range(0,len(data)):
    if data.iloc[j,3] in vis:
      continue
    val=fuzz.partial_ratio(data.iloc[i,3].lower(),data.iloc[j,3].lower())
    heapq.heappush(h,(-val,data.iloc[j,3],data.iloc[j,0]))
  c=0
  p=[]
  p.append(data.iloc[i,3]+'; '+data.iloc[i,0])
  c1=[]
  while(h):
    c+=1
    val1,val2,val3=heapq.heappop(h)
    val1=-val1
    if val1<threshold:
      break
    vis.add(val2)
    c1.append(str(val1)+"; "+ val2+ "; " +val3)
  p.append(c1)
  ans.append(p)
end = time.time()
print("The time of execution of above program is :", end-start)



The time of execution of above program is : 98.31156015396118


In [ ]:
fuzz_partial_ratio=pd.DataFrame(ans)
fuzz_partial_ratio.head(100)

,0,1
0,AERO-CARIBBEAN; 36,[]
1,AVIA IMPORT; 173,"[91; SEGOVIA IMPORT AND EXPORT CORP.; 19161, 82; DA'IL IMPORT AND EXPORT; 34984, 82; EMPRESA CUBANA IMPORTADORA DE COMBUSTIBLES Y LUBRICANTES; 26934, 82; EMPRESA CUBANA IMPORTADORA Y EXPORTADORA DE COMBUSTIBLES Y LUBRICANTES; 26934]"
2,NATIONAL BANK OF CUBA; 306,"[86; NATIONAL BANK OF IRAN; 25578, 81; CENTRAL NATIONAL BANK OF THE DONETSK PEOPLE'S REPUBLIC; 22486]"
3,COIBA; 540,"[89; TRANSCIBA; 11875, 80; AKTSIONERNY KOMMERCHESKI BANK MOSKOVSKI OBLASTNOI BANK OTKRYTOE AKTSIONERNOE OBSHCHESTVO; 19036, 80; AL-KHOSIBAN, Fahad H.A.; 10475, 80; BANCO BANDES; 26587, 80; COMBATANT FORCE FOR THE LIBERATION OF RWANDA; 15638]"
4,CRYMSA; 552,"[83; CRYSTAL; 15037, 83; M.E.N.A. CRYSTAL SUGAR COMPANY; 26892, 83; OBSHCHESTVO S OGRANICHENNOI OTVETSTVENNOSTYU 'KRYMSKAYA PERVAYA STRAKHOVAYA KOMPANIYA'; 22403, 83; OOO 'KRYMSKAYA PERVAYA STRAKHOVAYA KOMPANIYA'; 22403, 83; SONG, Crystal; 33274]"
5,COPROVA; 555,"[100; COPROVA SARL; 555, 86; PAREDEZ CORDOVA, Jorge Mario; 10382]"
6,CIMEX; 559,"[100; CIMEX CUBA; 8125, 100; FINCIMEX; 30630, 80; CENTER FOR THE STUDY OF ORGANIZED CRIME; 15555, 80; CRIMEAN REPUBLICAN ENTERPRISE AZOV DISTILLERY; 18966, 80; CRIMEAN REPUBLICAN ENTERPRISE AZOVSKY LIKEROVODOCHNY ZAVOD; 18966]"
7,DELVEST HOLDING COMPANY; 651,"[87; CASTLE INVEST HOLDING COMPANY SAL; 29024, 85; REDIN COMPANY; 27123, 84; BONYAD TOURISM AND RECREATIONAL CENTERS HOLDING COMPANY; 30401, 83; KHITI HOLDING COMPANY; 30163, 83; OAO INTEGRAL - MANAGEMENT HOLDING COMPANY INTEGRAL; 34981]"
8,CUBANA AIRLINES; 735,"[86; NESA; 15073, 85; DENA AIRLINES; 24529]"
9,CARIBBEAN EXPORT ENTERPRISE; 751,[]


### Token Sort Ratio


In token sort ratio, the strings are tokenized and pre-processed by converting to lower case and getting rid of punctuation. The strings are then sorted alphabetically and joined together. Post this, the Levenshtein distance similarity ratio is calculated between the strings. This comes in handy when the strings you are comparing are the same in spelling but are not in the same order. Let’s use another name example.

We apply the same procedure that we applied for Fuzz Ratio.



In [20]:
#TOKEN SORT Ratio
import heapq
threshold=80
ans=[]
vis=set()
start = time.time()
for i in range (0,20):
  h=[]
  if data.iloc[i,3] in vis:
    continue
  vis.add(data.iloc[i,3])  
  for j in range(0,len(data)):
    if data.iloc[j,3] in vis:
      continue
    val=fuzz.token_sort_ratio(data.iloc[i,3],data.iloc[j,3])
    heapq.heappush(h,(-val,data.iloc[j,3],data.iloc[j,0]))
  c=0
  p=[]
  p.append(data.iloc[i,3]+'; '+data.iloc[i,0])
  c1=[]
  while(h):
    c+=1
    val1,val2,val3=heapq.heappop(h)
    val1=-val1
    if val1<threshold:
      break
    vis.add(val2)
    c1.append(str(val1)+"; "+ val2+ "; " +val3)
  p.append(c1)
  ans.append(p)
end = time.time()
print("The time of execution of above program is :", end-start)


The time of execution of above program is : 62.14878988265991


In [21]:
fuzz_token_sort_ratio=pd.DataFrame(ans)
fuzz_token_sort_ratio.head(100)

,0,1
0,AERO-CARIBBEAN; 36,[]
1,AVIA IMPORT; 173,[]
2,NATIONAL BANK OF CUBA; 306,[86; NATIONAL BANK OF IRAN; 25578]
3,COIBA; 540,[]
4,CRYMSA; 552,[]
5,COPROVA; 555,[]
6,COPROVA SARL; 555,[]
7,CIMEX; 559,[]
8,DELVEST HOLDING COMPANY; 651,[]
9,CUBANA AIRLINES; 735,[]


### TOKEN SET RATIO


Token set ratio performs a set operation that takes out the common tokens instead of just tokenizing the strings, sorting, and then pasting the tokens back together. Extra or same repeated words do not matter. After this, it works just like token_sort_ratio. This function is the most helpful when applied to a set of strings with a significant difference in lengths.

We apply the same procedure that we applied for Fuzz Ratio.

In [22]:
#TOKEN SORT Ratio
import heapq
threshold=80
ans=[]
vis=set()
start = time.time()
for i in range (0,20):
  h=[]
  if data.iloc[i,3] in vis:
    continue
  vis.add(data.iloc[i,3])  
  for j in range(0,len(data)):
    if data.iloc[j,3] in vis:
      continue
    val=fuzz.token_set_ratio(data.iloc[i,3],data.iloc[j,3])
    heapq.heappush(h,(-val,data.iloc[j,3],data.iloc[j,0]))
  c=0
  p=[]
  p.append(data.iloc[i,3]+'; '+data.iloc[i,0])
  c1=[]
  while(h):
    c+=1
    val1,val2,val3=heapq.heappop(h)
    val1=-val1
    if val1<threshold:
      break
    vis.add(val2)
    c1.append(str(val1)+"; "+ val2+ "; " +val3)
  p.append(c1)
  ans.append(p)
end = time.time()
print("The time of execution of above program is :", end-start)


The time of execution of above program is : 59.29887580871582


In [23]:
fuzz_token_set_ratio=pd.DataFrame(ans)
fuzz_token_set_ratio.head(100)

,0,1
0,AERO-CARIBBEAN; 36,[]
1,AVIA IMPORT; 173,[]
2,NATIONAL BANK OF CUBA; 306,"[86; CENTRAL NATIONAL BANK OF THE DONETSK PEOPLE'S REPUBLIC; 22486, 86; NATIONAL BANK OF IRAN; 25578]"
3,COIBA; 540,[]
4,CRYMSA; 552,[]
5,COPROVA; 555,[100; COPROVA SARL; 555]
6,CIMEX; 559,[100; CIMEX CUBA; 8125]
7,DELVEST HOLDING COMPANY; 651,"[86; WOLF HOLDING COMPANY; 22554, 83; KHITI HOLDING COMPANY; 30163]"
8,CUBANA AIRLINES; 735,[]
9,CARIBBEAN EXPORT ENTERPRISE; 751,[]


Now we will apply all the techniques that we had run above on the grouped data that we had created earlier. Essentially, for each key, all the entries associated with that key will get tested against each other.

We measure the performance of the technique being used by dividing the actual number of matches by the potential number of matches (n-1)

In each group the first element is being tested against all the other  elements in the group.

### FUZZ RATIO ON GROUPED DATA

In [24]:
ans=[]
threshold=80
start = time.time()
for i in group:
  if len(group[i])==1:
    continue
  curr=group[i][0]
  p=[]
  co=0
  for j in range(1,len(group[i])):
      val=fuzz.ratio(curr,group[i][j])
      if val>threshold:
        co+=1
      p.append((group[i][j],val))
  p=sorted(p,key=lambda x:-x[1])
  ans.append([curr])
  ans[-1].append(co/(len(group[i])-1)*100)
  ans[-1].append(p)    

end = time.time()
print("The time of execution of above program is :", end-start)  


The time of execution of above program is : 0.6336572170257568


In [25]:
fuzz_ratio1=pd.DataFrame(ans)
fuzz_ratio1.head(100)

print("Performance of FUZZ RATIO", fuzz_ratio1.iloc[:,1].sum()/len(fuzz_ratio1))

Performance of FUZZ RATIO 28.735155694929905


### FUZZ PARTIAL RATIO

In [26]:
ans=[]
threshold=80
start = time.time()
for i in group:
  if len(group[i])==1:
    continue
  curr=group[i][0]
  p=[]
  co=0
  for j in range(1,len(group[i])):
      val=fuzz.partial_ratio(curr,group[i][j])
      if val>threshold:
        co+=1
      p.append((group[i][j],val))
  p=sorted(p,key=lambda x:-x[1])
  ans.append([curr])
  ans[-1].append(co/(len(group[i])-1)*100)
  ans[-1].append(p)    

end = time.time()
print("The time of execution of above program is :", end-start)  


The time of execution of above program is : 2.6530191898345947


In [27]:
fuzz_partial_ratio1=pd.DataFrame(ans)
fuzz_partial_ratio1.head(100)

print("Performance of FUZZ PARTIAL RATIO", fuzz_partial_ratio1.iloc[:,1].sum()/len(fuzz_partial_ratio1))

Performance of FUZZ PARTIAL RATIO 41.91531886916336


### TOKEN SORT RATIO

In [28]:
ans=[]
threshold=80
start = time.time()
for i in group:
  if len(group[i])==1:
    continue
  curr=group[i][0]
  p=[]
  co=0
  for j in range(1,len(group[i])):
      val=fuzz.token_sort_ratio(curr,group[i][j])
      if val>threshold:
        co+=1
      p.append((group[i][j],val))
  p=sorted(p,key=lambda x:-x[1])
  ans.append([curr])
  ans[-1].append(co/(len(group[i])-1)*100)
  ans[-1].append(p)    

end = time.time()
print("The time of execution of above program is :", end-start)  


The time of execution of above program is : 0.6790406703948975


In [29]:
fuzz_token_sort_ratio1=pd.DataFrame(ans)
fuzz_token_sort_ratio1.head(100)

print("Performance of TOKEN SORT RATIO", fuzz_token_sort_ratio1.iloc[:,1].sum()/len(fuzz_token_sort_ratio1))

Performance of TOKEN SORT RATIO 27.96020814867008


### TOKEN SET RATIO

In [30]:
ans=[]
threshold=80
start = time.time()
for i in group:
  if len(group[i])==1:
    continue
  curr=group[i][0]
  p=[]
  co=0
  for j in range(1,len(group[i])):
      val=fuzz.token_set_ratio(curr,group[i][j])
      if val>threshold:
        co+=1
      p.append((group[i][j],val))
  p=sorted(p,key=lambda x:-x[1])
  ans.append([curr])
  ans[-1].append(co/(len(group[i])-1)*100)
  ans[-1].append(p)    

end = time.time()
print("The time of execution of above program is :", end-start)  


The time of execution of above program is : 1.0010173320770264


In [31]:
fuzz_token_set_ratio1=pd.DataFrame(ans)
fuzz_token_set_ratio1.head(100)

print("Performance of TOKEN SET RATIO", fuzz_token_set_ratio1.iloc[:,1].sum()/len(fuzz_token_set_ratio1))

Performance of TOKEN SET RATIO 41.599672343617925


## OTHER DISTANCES

In [32]:
pip install recordlinkage


     |████████████████████████████████| 926 kB 7.0 MB/s 
     |████████████████████████████████| 132 kB 74.3 MB/s 
  Created wheel for jellyfish: filename=jellyfish-0.9.0-cp37-cp37m-linux_x86_64.whl size=73991 sha256=bc223b09883ff25471c0fb1c6bcdca123c1d9f397d7f7a853a45da267dfe4f27
  Stored in directory: /root/.cache/pip/wheels/fe/99/4e/646ce766df0d070b0ef04db27aa11543e2767fda3075aec31b
Successfully built jellyfish


In [33]:
import recordlinkage

Below we have created a function which computes the jaro distance between two strings. 

In [34]:
from math import floor, ceil
def jaro_distance(s1, s2):
    if (s1 == s2):
        return 1.0
    len1 = len(s1)
    len2 = len(s2)
    max_dist = floor(max(len1, len2) / 2) - 1
    match = 0
    hash_s1 = [0] * len(s1)
    hash_s2 = [0] * len(s2)
    for i in range(len1):
        for j in range(max(0, i - max_dist),
                       min(len2, i + max_dist + 1)):
            if (s1[i] == s2[j] and hash_s2[j] == 0):
                hash_s1[i] = 1
                hash_s2[j] = 1
                match += 1
                break
    if (match == 0):
        return 0.0
    t = 0
    point = 0
    for i in range(len1):
        if (hash_s1[i]):
            while (hash_s2[point] == 0):
                point += 1
            if (s1[i] != s2[point]):
                t += 1
            point += 1
    t = t//2
    return round(((match/ len1 + match / len2 +
            (match - t) / match)/ 3.0)*100,2)
 

Now for each of the groups we check the jaro similarity of the items present in the group. If the similarity score is above 80, we count it as a match. To deterine how well the algorithm works, we divide the number of actual matches by the number of potential matches

JARO SIMILARITY


In [35]:
ans=[]
threshold=80
start = time.time()
for i in group:
  if len(group[i])==1:
    continue
  curr=group[i][0]
  p=[]
  co=0
  for j in range(1,len(group[i])):
      val=jaro_distance(curr,group[i][j])
      if val>threshold:
        co+=1
      p.append((group[i][j],val))
  p=sorted(p,key=lambda x:-x[1])
  ans.append([curr])
  ans[-1].append(co/(len(group[i])-1)*100)
  ans[-1].append(p)    

end = time.time()
print("The time of execution of above program is :", end-start)  


The time of execution of above program is : 0.4520590305328369


In [36]:
JARO=pd.DataFrame(ans)
JARO.head(100)

print("Performance of JARO", JARO.iloc[:,1].sum()/len(JARO))

Performance of JARO 40.870055508645514


In [37]:
JARO.head(100)

,0,1,2
0,COPROVA,100.000000,"[(COPROVA SARL, 86.11)]"
1,CUFLET,0.000000,"[(CUBAN FREIGHT ENTERPRISE, 68.06)]"
2,"AL-ZAWAHIRI, Ayman",50.000000,"[(AL-ZAWAHIRI, Aiman Muhammad Rabi, 82.52), (SALIM, Ahmad Fuad, 55.5)]"
3,"ZUMAR, Abbud",25.000000,"[(AL-ZUMAR, Abbud, 82.22), (AL-ZUMUR, Abood Abdul Latif Hassan, 70.1), (AL-ZAMUR, Abboud Abdul Latif Hassan, 66.98), (EL-ZOMOR, Aboud Abdul Latif Hasan, 62.88)]"
4,"HAWATMEH, Nayif",50.000000,"[(HAWATMAH, Nayif, 95.56), (KHALID, Abu, 52.93)]"
5,MIDCO FINANCIAL S.A.,0.000000,"[(MONTANA MANAGEMENT INC., 56.84)]"
6,"MARZUK, Musa Abu",50.000000,"[(MARZOUK, Musa Abu, 98.04), (MARZOOK, Mousa Mohamed Abou, 83.1), (ABU-MARZUQ, Dr. Musa, 66.7), (ABU-MARZUQ, Sa'id, 57.35)]"
7,"ABDULLAH, Dr. Ramadan",0.000000,"[(SHALLAH, Dr. Ramadan Abdullah, 78.89), (SHALLAH, Ramadan Abdalla Mohamed, 59.43)]"
8,CENTRAL BANK OF THE ISLAMIC REPUBLIC OF IRAN,0.000000,"[(CENTRAL BANK OF IRAN, 77.64), (BANK MARKAZI IRAN, 57.1)]"
9,WORKERS' WELFARE BANK - OF IRAN,0.000000,"[(BANK REFAH, 54.67)]"


In [38]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [39]:
pip install ftfy

     |████████████████████████████████| 53 kB 1.6 MB/s 


Below we will remove all the punctuation and then create Ngrams (length of 3)of the different strings in the input column. The Ngrams of each string becomes a list


In [40]:
import re
from ftfy import fix_text
def ngrams(string, n=3):
    string = fix_text(string)
    string = string.encode("ascii", errors="ignore").decode() 
    string = string.lower()
    chars_to_remove = [")","(",".","|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.title() 
    string = re.sub(' +',' ',string).strip() 
    string = ' '+ string +' '
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

Example

In [41]:

ngrams(data.iloc[0,3])

[' Ae',
 'Aer',
 'ero',
 'ro ',
 'o C',
 ' Ca',
 'Car',
 'ari',
 'rib',
 'ibb',
 'bbe',
 'bea',
 'ean',
 'an ']

In [42]:
input_clean = data['Input'].unique()


vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams, lowercase=False)

tfidf = vectorizer.fit_transform(input_clean)



In [43]:
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=5, n_jobs=-1).fit(tfidf)


In [49]:
unique_org= data['Input']

In the below function, the list of Ngrams is converted into a list of vectors using TFIDF. Once all the lists of Ngrams have been converted into vectors, we see the 5 closest such vectors to the vector that is being tested.

In [50]:
def getNearestN(query):
    queryTFIDF_ = vectorizer.transform(query)
    distances, indices = nbrs.kneighbors(queryTFIDF_)
    return distances, indices

distances, indices = getNearestN(unique_org)
matches = []
for i,j in enumerate(indices):
    temp=[]
    temp.append(unique_org[i])
    c1=[]
    for i1,j1 in enumerate(j):
      c1.append( [round(distances[i][i1],2), input_clean[j1]])
    temp.extend(c1)  
    matches.append(temp)


matches = pd.DataFrame(matches)



In [51]:
matches.head(10)

,0,1,2,3,4,5
0,AERO-CARIBBEAN,"[0.0, AERO-CARIBBEAN]","[0.98, CARIBBEAN EXPORT ENTERPRISE]","[1.11, AERO CONTINENT S.A.]","[1.14, AERO CONTINENTE, INC.]","[1.19, AERO SKY ONE LTD]"
1,AVIA IMPORT,"[0.0, AVIA IMPORT]","[0.93, SEGOVIA IMPORT AND EXPORT CORP.]","[1.0, MDS IMPORT EXPORT]","[1.01, DA'IL IMPORT AND EXPORT]","[1.03, RUSSIAN EXPORT-IMPORT BANK]"
2,NATIONAL BANK OF CUBA,"[0.0, NATIONAL BANK OF CUBA]","[0.82, NATIONAL BANK OF IRAN]","[1.04, ISLAMIC NATIONAL BANK COMPANY]","[1.09, PALESTINE ISLAMIC NATIONAL BANK]","[1.12, CENTRAL BANK OF IRAN]"
3,COIBA,"[0.0, COIBA]","[1.1, LASHKAR E-TOIBA]","[1.18, COINTERANDINA]","[1.21, COINEMP S.A.]","[1.23, TRANSCIBA]"
4,CRYMSA,"[0.0, CRYMSA]","[1.12, CRYSTAL]","[1.18, SONG, Crystal]","[1.25, M.E.N.A. CRYSTAL SUGAR COMPANY]","[1.27, AL-ZUBAYDI, Muhammad Hamsa]"
5,COPROVA,"[0.0, COPROVA]","[0.66, COPROVA SARL]","[1.19, COPETE S.A.]","[1.2, ZHIROVA, Elena Borisnova]","[1.26, IMENI KADYROVA, OF]"
6,COPROVA SARL,"[0.0, COPROVA SARL]","[0.66, COPROVA]","[1.13, MMT SARL]","[1.16, TAJCO SARL]","[1.2, LANDMATICS SARL]"
7,CIMEX,"[0.0, CIMEX]","[0.75, CIMEX CUBA]","[1.06, FINCIMEX]","[1.27, REMEXI]","[1.28, CIFUENTES VILLA, Alex]"
8,DELVEST HOLDING COMPANY,"[0.0, DELVEST HOLDING COMPANY]","[0.93, CASTLE INVEST HOLDING COMPANY SAL]","[0.96, SHAM HOLDING COMPANY SAL]","[0.97, KHITI HOLDING COMPANY]","[1.01, WOLF HOLDING COMPANY]"
9,CUBANA AIRLINES,"[0.0, CUBANA AIRLINES]","[0.79, DENA AIRLINES]","[0.92, POUYA AIRLINES]","[0.97, SHAM WING AIRLINES]","[0.99, ALNASER AIRLINES]"
